## **1. Datasest Creation**

In [ ]:
import os

# find the dataset here: https://www.unb.ca/cic/datasets/invesandmal2019.html
!wget http://205.174.165.80/CICDataset/CICInvesAndMal2019/Dataset/APKs/Adware.zip
!wget http://205.174.165.80/CICDataset/CICInvesAndMal2019/Dataset/APKs/PremiumSMS.zip
!wget http://205.174.165.80/CICDataset/CICInvesAndMal2019/Dataset/APKs/Ransomware.zip
!wget http://205.174.165.80/CICDataset/CICInvesAndMal2019/Dataset/APKs/SMS.zip
!wget http://205.174.165.80/CICDataset/CICInvesAndMal2019/Dataset/APKs/Scareware.zip

!wget http://205.174.165.80/CICDataset/CICInvesAndMal2019/Dataset/APKs/Benign_2015.zip
!wget http://205.174.165.80/CICDataset/CICInvesAndMal2019/Dataset/APKs/Benign_2016.zip
!wget http://205.174.165.80/CICDataset/CICInvesAndMal2019/Dataset/APKs/Benign_2017.zip

!unzip /content/Adware.zip
os.remove('/content/Adware.zip')
!unzip /content/PremiumSMS.zip
os.remove('/content/PremiumSMS.zip')
!unzip /content/SMS.zip
os.remove('/content/SMS.zip')
!unzip /content/Ransomware.zip
os.remove('/content/Ransomware.zip')
!unzip /content/Scareware.zip
os.remove('/content/Scareware.zip')

!unzip /content/Benign_2015.zip
os.remove('/content/Benign_2015.zip')
!unzip /content/Benign_2016.zip
os.remove('/content/Benign_2016.zip')
!unzip /content/Benign_2017.zip
os.remove('/content/Benign_2017.zip')

In [ ]:
import zipfile

#zip 파일 풀기
for root, dirs, files in os.walk('/content/mal'):
  for file in files:
    if file.endswith('.zip'):
      try:
        with zipfile.ZipFile(os.path.join(root,file), 'r') as zip_ref:
          zip_ref.extractall()
        os.remove(os.path.join(root,file))
      except:
        os.remove(os.path.join(root,file))
for root, dirs, files in os.walk('/content/mal'):
  for file in files:
    if file.endswith('.apk'):
      print(file)
      shutil.copy(os.path.join(root,file), '/content/dataset/malign/')
      os.remove(os.path.join(root,file))

In [ ]:
import os, sys, stat 

# malign, benign 개수 구하기
malign = '/content/drive/MyDrive/Dataset/dataset/malign'
benign = '/content/drive/MyDrive/Dataset/dataset/benign'
countM = 0
countB = 0

for entry in os.scandir(benign):
  if entry.path.endswith(".apk") and entry.is_file():
          countB += 1
for entry in os.scandir(malign):
  if entry.path.endswith(".apk") and entry.is_file():
          countM += 1
          
print('benign : ', countB)
print('malign : ', countM)

In [ ]:
!pip install androguard

In [ ]:
import os, sys, stat 
import warnings
from androguard.core.bytecodes.apk import APK
warnings.filterwarnings("ignore")

# androguard 로 apk permission 추출
malign = '/content/drive/MyDrive/Dataset/dataset/malign'
benign = '/content/drive/MyDrive/Dataset/dataset/benign'

permissions =  []
countM = 0
countB = 0

for entry in os.scandir(benign):
  if entry.path.endswith(".apk") and entry.is_file():
          try:
            a = APK(entry.path)
            perm = a.get_permissions()
            countB += 1
            for per in perm:
              if per not in permissions and per.startswith('android.permission'):
                permissions.append(per)
          except:
            os.remove(entry.path)
  print('benign : ', perm)          

for entry in os.scandir(malign):
  if entry.path.endswith(".apk") and entry.is_file():
          try:
            a = APK(entry.path)
            perm = a.get_permissions()
            countM += 1
            for per in perm:
              if per not in permissions and per.startswith('android.permission'):
                permissions.append(per)
          except:
            os.remove(entry.path)
  print('malign : ', perm)

In [5]:
import pandas as pd
default = open('/content/drive/My Drive/android_permissions.txt','r').readlines()
perms = [s.rstrip('\n') for s in default]
for p in permissions:
  if p not in perms:
    perms.append(p)
print(len(perms))

with open('/content/drive/My Drive/use_permissions.txt','w+') as f:
  for p in perms:
    f.write(p + '\n')

pe = []
with open('/content/drive/My Drive/use_permissions.txt','r') as a:
  content = a.readlines()
  for line in content:
    cur_perm = line[:-1]
    pe.append(cur_perm)
len(pe)
print(pe)

428
['android.permission.ACCESS_ALL_DOWNLOADS', 'android.permission.ACCESS_BLUETOOTH_SHARE', 'android.permission.ACCESS_CACHE_FILESYSTEM', 'android.permission.ACCESS_CHECKIN_PROPERTIES', 'android.permission.ACCESS_CONTENT_PROVIDERS_EXTERNALLY', 'android.permission.ACCESS_DOWNLOAD_MANAGER', 'android.permission.ACCESS_DOWNLOAD_MANAGER_ADVANCED', 'android.permission.ACCESS_DRM_CERTIFICATES', 'android.permission.ACCESS_EPHEMERAL_APPS', 'android.permission.ACCESS_FM_RADIO', 'android.permission.ACCESS_INPUT_FLINGER', 'android.permission.ACCESS_KEYGUARD_SECURE_STORAGE', 'android.permission.ACCESS_LOCATION_EXTRA_COMMANDS', 'android.permission.ACCESS_MOCK_LOCATION', 'android.permission.ACCESS_MTP', 'android.permission.ACCESS_NETWORK_CONDITIONS', 'android.permission.ACCESS_NETWORK_STATE', 'android.permission.ACCESS_NOTIFICATIONS', 'android.permission.ACCESS_NOTIFICATION_POLICY', 'android.permission.ACCESS_PDB_STATE', 'android.permission.ACCESS_SURFACE_FLINGER', 'android.permission.ACCESS_VOICE_I

In [ ]:
# permission -> csv

dataset_df = pd.DataFrame(columns=perms)
dataset_b = {}
id = 0
for entry in os.scandir(benign):
  if entry.path.endswith(".apk") and entry.is_file():
    a = APK(entry.path)
    perm = a.get_permissions()
    id += 1
    for d in perms:
      if d in perm:
        dataset_b[d]=1
      else:
        dataset_b[d]=0
    dataset_b['class'] = 'benign'
    dataset_df=dataset_df.append(dataset_b, ignore_index=True)
dataset_m = {}
for entry in os.scandir(malign):
  if entry.path.endswith(".apk") and entry.is_file():
    a = APK(entry.path)
    perm = a.get_permissions()
    id += 1
    for d in perms:
      if d in perm:
        dataset_m[d]=1
      else:
        dataset_m[d]=0
    dataset_m['class'] = 'malign'
    dataset_df=dataset_df.append(dataset_m, ignore_index=True)

dataset_df.to_csv('/content/drive/My Drive/Dataset/dataset/android_dataset.csv', index=False)
   

## **2. SVM : SVC with Grid Search to tune model**

In [11]:
import sklearn
import pickle
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix 
from androguard.core.bytecodes.apk import APK
import warnings
warnings.filterwarnings(action='ignore')

Y = dataset_df['class']
X = dataset_df.drop(['class'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

# defining parameter range  - 후에 더 정밀한 값 넣기
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
			'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
			'kernel': ['rbf']} 

grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 1) 

# fitting the model for grid search 
grid.fit(X_train, y_train) 
print(grid.best_params_) 

# predict
grid_predictions = grid.predict(X_test) 

# print classification report 
print(classification_report(y_test, grid_predictions)) 
pickle.dump(grid, open('/content/drive/MyDrive/svc_new.pkl', 'wb'))

def predict(apk):
  vector = {}
  a = APK(apk)
  perm = a.get_permissions()
  print(perm)
  for d in perms:
    if d in perm:
      vector[d]=1
    else:
      vector[d]=0
  input = [ v for v in vector.values() ]
  print(input)
  print('result :', grid.predict([input]))

# test
# malign apk
predict('/content/drive/MyDrive/Dataset/1c53e2c34d1219a2fae8fcf8ec872ac8.apk')
# benign apk
predict('/content/drive/MyDrive/Dataset/a.envisionmobile.caa.apk')

Fitting 5 folds for each of 25 candidates, totalling 125 fits
{'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
              precision    recall  f1-score   support

      benign       0.89      0.95      0.92       149
      malign       0.90      0.81      0.85        94

    accuracy                           0.89       243
   macro avg       0.90      0.88      0.88       243
weighted avg       0.89      0.89      0.89       243

['android.permission.SYSTEM_ALERT_WINDOW', 'android.permission.READ_CONTACTS', 'android.permission.WRITE_EXTERNAL_STORAGE', 'android.permission.READ_PROFILE', 'android.permission.READ_PHONE_STATE', 'android.permission.CAMERA', 'android.permission.READ_EXTERNAL_STORAGE', 'android.permission.ACCESS_NETWORK_STATE', 'android.permission.WAKE_LOCK', 'com.android.browser.permission.READ_HISTORY_BOOKMARKS', 'android.permission.GET_ACCOUNTS', 'android.permission.RECEIVE_BOOT_COMPLETED', 'android.permission.GET_TASKS', 'android.permission.VIBRATE', 'android.permission.WRITE

## **3. RandomForest**

In [12]:
import numpy as np
import random
import pickle
import warnings
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from androguard.core.bytecodes.apk import APK
warnings.filterwarnings(action='ignore')

feature_vector = []

with open('/content/drive/My Drive/Dataset/dataset/android_dataset.csv','r') as fp:
	for i,line in enumerate(fp):
		if i == 0:
			pass
		else:
			feature_vector.append([x.strip() for x in line.split(',')])

random.shuffle(feature_vector)

X = [x[:-1] for x in feature_vector]
Y = [y[-1] for y in feature_vector]

# train : test = 8: 2
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8, test_size=0.2, random_state=0)

# defining parameter range  - 후에 더 정밀한 값 넣기
rf = RandomForestClassifier(n_estimators=1000, random_state=0, criterion="entropy")

# fitting the model
rf.fit(X_train,y_train)

# predict
y_pred=rf.predict(X_test)
rf_predictions = rf.predict(X_test) 

# print classification report 
print(classification_report(y_test, rf_predictions)) 
pickle.dump(rf, open('/content/drive/MyDrive/randomfrorest_new.pkl', 'wb'))

def predict(apk):
  vector = {}
  a = APK(apk)
  perm = a.get_permissions()
  print(perm)
  for d in perms:
    if d in perm:
      vector[d]=1
    else:
      vector[d]=0
  input = [ v for v in vector.values() ]
  print(input)
  print('result :',rf.predict([input]))

# test
# malign apk
predict('/content/drive/MyDrive/Dataset/1c53e2c34d1219a2fae8fcf8ec872ac8.apk')
# malign apk
predict('/content/drive/MyDrive/Dataset/a.envisionmobile.caa.apk')

              precision    recall  f1-score   support

      benign       0.93      0.98      0.96       159
      malign       0.96      0.87      0.91        84

    accuracy                           0.94       243
   macro avg       0.95      0.93      0.93       243
weighted avg       0.94      0.94      0.94       243

['android.permission.SYSTEM_ALERT_WINDOW', 'android.permission.READ_CONTACTS', 'android.permission.WRITE_EXTERNAL_STORAGE', 'android.permission.READ_PROFILE', 'android.permission.READ_PHONE_STATE', 'android.permission.CAMERA', 'android.permission.READ_EXTERNAL_STORAGE', 'android.permission.ACCESS_NETWORK_STATE', 'android.permission.WAKE_LOCK', 'com.android.browser.permission.READ_HISTORY_BOOKMARKS', 'android.permission.GET_ACCOUNTS', 'android.permission.RECEIVE_BOOT_COMPLETED', 'android.permission.GET_TASKS', 'android.permission.VIBRATE', 'android.permission.WRITE_SETTINGS', 'android.permission.INTERNET']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,

## **4. XGBoost**

In [13]:
import xgboost as xgb
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
import pickle
import warnings
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from androguard.core.bytecodes.apk import APK
warnings.filterwarnings(action='ignore')

feature_vector = []

with open('/content/drive/My Drive/Dataset/dataset/android_dataset.csv','r') as fp:
	for i,line in enumerate(fp):
		if i == 0:
			pass
		else:
			feature_vector.append([x.strip() for x in line.split(',')])

X = [x[:-1] for x in feature_vector]
Y = [y[-1] for y in feature_vector]

# Fit the classifier to the training set
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8, test_size=0.2, random_state=0)
X_train=np.array(X_train)
y_train=np.array(y_train)
X_test=np.array(X_test)

# defining parameter range  - 후에 더 정밀한 값 넣기
xgb = XGBClassifier(max_depth=10, learning_rate=1, n_estimators=100)

# fitting the model
xgb.fit(X_train,y_train)

# predict
y_pred=xgb.predict(X_test)
xgb_predictions = xgb.predict(X_test)

# print classification report 
print(classification_report(y_test, xgb_predictions)) 
pickle.dump(xgb, open('/content/drive/MyDrive/xgboost_new.pkl', 'wb'))

def predict(apk):
  vector = {}
  a = APK(apk)
  perm = a.get_permissions()
  print(perm)
  for d in perms:
    if d in perm:
      vector[d]=1
    else:
      vector[d]=0
  input = [ v for v in vector.values() ]
  print(input)
  print('result :',xgb.predict([input]))

# test
# malign apk
predict('/content/drive/MyDrive/Dataset/1c53e2c34d1219a2fae8fcf8ec872ac8.apk')
# malign apk
predict('/content/drive/MyDrive/Dataset/a.envisionmobile.caa.apk')

              precision    recall  f1-score   support

      benign       0.95      0.90      0.93       153
      malign       0.85      0.92      0.88        90

    accuracy                           0.91       243
   macro avg       0.90      0.91      0.90       243
weighted avg       0.91      0.91      0.91       243

['android.permission.SYSTEM_ALERT_WINDOW', 'android.permission.READ_CONTACTS', 'android.permission.WRITE_EXTERNAL_STORAGE', 'android.permission.READ_PROFILE', 'android.permission.READ_PHONE_STATE', 'android.permission.CAMERA', 'android.permission.READ_EXTERNAL_STORAGE', 'android.permission.ACCESS_NETWORK_STATE', 'android.permission.WAKE_LOCK', 'com.android.browser.permission.READ_HISTORY_BOOKMARKS', 'android.permission.GET_ACCOUNTS', 'android.permission.RECEIVE_BOOT_COMPLETED', 'android.permission.GET_TASKS', 'android.permission.VIBRATE', 'android.permission.WRITE_SETTINGS', 'android.permission.INTERNET']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,

## **5. Decision Tree**

In [15]:
import numpy as np
import random
import pickle
import warnings
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from androguard.core.bytecodes.apk import APK
warnings.filterwarnings(action='ignore')

feature_vector = []

with open('/content/drive/My Drive/Dataset/dataset/android_dataset.csv','r') as fp:
	for i,line in enumerate(fp):
		if i == 0:
			pass
		else:
			feature_vector.append([x.strip() for x in line.split(',')])

random.shuffle(feature_vector)

X = [x[:-1] for x in feature_vector]
Y = [y[-1] for y in feature_vector]

# train : test = 8: 2
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8, test_size=0.2, random_state=0)

# defining parameter range  - 후에 더 정밀한 값 넣기
decisiontree = DecisionTreeClassifier(random_state=0)

# fitting the model
decisiontree.fit(X_train,y_train)

# predict
y_pred=decisiontree.predict(X_test)
decisiontree_predictions = decisiontree.predict(X_test) 

# print classification report 
print(classification_report(y_test, decisiontree_predictions)) 
pickle.dump(decisiontree, open('/content/drive/MyDrive/decisiontree_new.pkl', 'wb'))

def predict(apk):
  vector = {}
  a = APK(apk)
  perm = a.get_permissions()
  print(perm)
  for d in perms:
    if d in perm:
      vector[d]=1
    else:
      vector[d]=0
  input = [ v for v in vector.values() ]
  print(input)
  print('result :',decisiontree.predict([input]))

# test
# malign apk
predict('/content/drive/MyDrive/Dataset/1c53e2c34d1219a2fae8fcf8ec872ac8.apk')
# malign apk
predict('/content/drive/MyDrive/Dataset/a.envisionmobile.caa.apk')

              precision    recall  f1-score   support

      benign       0.92      0.89      0.91       158
      malign       0.81      0.86      0.83        85

    accuracy                           0.88       243
   macro avg       0.87      0.88      0.87       243
weighted avg       0.88      0.88      0.88       243

['android.permission.SYSTEM_ALERT_WINDOW', 'android.permission.READ_CONTACTS', 'android.permission.WRITE_EXTERNAL_STORAGE', 'android.permission.READ_PROFILE', 'android.permission.READ_PHONE_STATE', 'android.permission.CAMERA', 'android.permission.READ_EXTERNAL_STORAGE', 'android.permission.ACCESS_NETWORK_STATE', 'android.permission.WAKE_LOCK', 'com.android.browser.permission.READ_HISTORY_BOOKMARKS', 'android.permission.GET_ACCOUNTS', 'android.permission.RECEIVE_BOOT_COMPLETED', 'android.permission.GET_TASKS', 'android.permission.VIBRATE', 'android.permission.WRITE_SETTINGS', 'android.permission.INTERNET']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,